In [ ]:
import numpy as np
import pandas as pd
from lsst.geom import Extent2I
from astropy.coordinates import SkyCoord
from lsst.cst.helpers import create_interactive_image, create_rgb_composite_image
from lsst.cst.data_visualization import DataImageDisplay, ScatterOptions

from lsst.cst.utilities import save_plot_as_html, delete_plot
from lsst.cst.image_display.interactors import HoverTool
from lsst.cst.helpers import create_linked_plot_with_brushing, create_datashader_plot, create_bounding_boxes_calexps_overlapping_a_point_plot, create_psf_flux_plot
from lsst.cst.conversions import data_id_to_str
from lsst.cst.utilities.queries import Band, ExposureDataHandler, DataWrapper, TAPService, QueryExposureData
from lsst.cst.conversions import nearest_patch_from_ra_dec, psf_size_at_pixel_xy
from lsst.cst import set_log_level
import logging

set_log_level(logging.DEBUG)
from holoviews import Scatter

Get data image and sources information from the butler

In [ ]:
from lsst.daf.butler import Butler, ButlerConfig
config = 'dp02'
collection = '2.2i/runs/DP0.2'
butler = Butler(config, collections=collection)

In [ ]:
calexpId = {'visit': 192350, 'detector': 175, 'band': 'i'}
calexp = butler.get('calexp', dataId=calexpId)
assert calexp is not None
calexp_sources = butler.get('sourceTable', dataId=calexpId)
assert calexp_sources is not None
print(calexp.getDimensions())

Create a plot with the image and the sources. Sources will be shown over the image with the hover tool enabled so that position will be shown when mouse is over. <br/>
Parameters available:<br/>
title: str<br/>
axes_label: Tuple[str, str]<br/>
marker: str<br/>
marker_color: str<br/>

In [ ]:
image_plot = create_interactive_image(calexp, calexp_sources, title=data_id_to_str(calexpId), axes_label=("X", "Y"))
image_plot

Saving plot:

In [ ]:
save_plot_as_html(image_plot, "image_plot.html")

Deleting plot:

In [ ]:
delete_plot(image_plot)

Get information from the TAP Service:

In [ ]:
from astropy import units as u
coord = SkyCoord(ra=55.65*u.degree, dec=-40.*u.degree, frame='icrs')
query = QueryExposureData.from_sky_coord(coord, 1.0)
tap_exposure_data = TAPService()
tap_exposure_data.query = query

In [ ]:
data = tap_exposure_data.fetch()

Create a linked plot out of a pandas Series with the **create_linked_plot_with_brushing** function, note that if no column is selected, first two columns of the series will be used for the plot.<br/> Parameters available: <br/>
data: pd.Series <br/>
columns: Tuple[str, str], optional<br/>
hovertool: bokeh.models.tools.HoverTool, optional<br/>

In [ ]:
raDecHover = HoverTool(
    tooltips=[
        ('ra,dec', '@coord_ra / @coord_dec'),
        ('rmag', '@mag_r_cModel'),
        ('type', '@shape_type'),
    ],
    formatters={
        'ra/dec': 'printf',
        'rmag': 'numeral',
        'type': 'printf',
    },
    point_policy="follow_mouse"
)
reduced_data = data.reduce_data(0.02)
linked_brush_plot = create_linked_plot_with_brushing(reduced_data, hovertool=raDecHover)

In [ ]:
linked_brush_plot

In [ ]:
save_plot_as_html(linked_brush_plot, "linked_brush_plot.html")

In [ ]:
delete_plot(linked_brush_plot)

Create datashader plot out of a pandas Series using the **create_datashader_plot** function, note that if no column is selected, first two columns of the series will be used for the plot.<br/> Parameters available: <br/>
data: Union[lsst.cst.data.tools.ExposureData, array],<br/>
columns: Optional[Tuple[str, str]] = None <br/>

In [ ]:
datashader_plot = create_datashader_plot(data, ("gmr", "gmi"))

In [ ]:
datashader_plot

In [ ]:
save_plot_as_html(datashader_plot, "datashader_plot.html")

In [ ]:
delete_plot(datashader_plot)

In [ ]:
my_ra_deg = 55.745834
my_dec_deg = -32.269167
values = tract_patch_from_ra_dec(my_ra_deg, my_dec_deg)

In [ ]:
print(values)

In [ ]:
values.tract, values.patch

In [ ]:
coord = SkyCoord(62.0, -37.0, frame='icrs', unit='deg')
mjd_range = (60250, 60300)
create_bounding_boxes_calexps_overlapping_a_point_plot(coord, mjd_range)

In [ ]:
create_psf_flux_plot(1252220598734556212, Band.i)

In [ ]:
datasetType = 'calexp'
dataId = {'visit': 192350, 'detector': 175}
calexp = butler.get(datasetType, dataId=dataId)

In [ ]:
from lsst.geom import Point2D, radToDeg, SpherePoint, degrees
point_tuple = (2000, 3500)
point_image = Point2D(point_tuple)
info_calexp = calexp.getInfo()
psf_calexp = info_calexp.getPsf()
bbox_calexp = calexp.getBBox()
props = psf_size_at_pixel_xy(psf_calexp, bbox_calexp, point_tuple)
print(props)

In [ ]:
create_rgb_composite_image(butler, 53.5693184520789,-34.57214410322362)